<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-и-подготовка-данных" data-toc-modified-id="Загрузка-и-подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка и подготовка данных</a></span></li><li><span><a href="#Обучение-и-проверка-модели" data-toc-modified-id="Обучение-и-проверка-модели-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение и проверка модели</a></span></li><li><span><a href="#Подготовка-к-расчёту-прибыли" data-toc-modified-id="Подготовка-к-расчёту-прибыли-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Подготовка к расчёту прибыли</a></span></li><li><span><a href="#Расчёта-прибыли-по-выбранным-скважинам" data-toc-modified-id="Расчёта-прибыли-по-выбранным-скважинам-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Расчёта прибыли по выбранным скважинам</a></span></li><li><span><a href="#Расчёт-прибыли-и-рисков" data-toc-modified-id="Расчёт-прибыли-и-рисков-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Расчёт прибыли и рисков</a></span></li></ul></div>

# Выбор локации для скважины

**Заказчик**

Добывающей компании «ГлавРосГосНефть».

**Задача**

Построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализировать возможную прибыль и риски.

**Данные**

Пробы нефти в трёх регионах: в каждом 100000 месторождений, где измерили качество нефти и объём её запасов. 

## Загрузка и подготовка данных

In [1]:
# загрузка библиотек  
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from scipy import stats as st

In [2]:
#сохраняем путь к файлу на сервере Практикума
server_path_geo_0 = '/datasets/geo_data_0.csv' 
server_path_geo_1 = '/datasets/geo_data_1.csv'
server_path_geo_2 = '/datasets/geo_data_2.csv'

#сохраняем путь к файлу на компьютере
local_path_geo_0 = 'datasets/geo_data_0.csv'  
local_path_geo_1 = 'datasets/geo_data_1.csv' 
local_path_geo_2 = 'datasets/geo_data_2.csv'


try:                                                   #пробуем найти данные по сетевому адресу
    geo_0 = pd.read_csv(server_path_geo_0) 
    geo_1 = pd.read_csv(server_path_geo_1)
    geo_2 = pd.read_csv(server_path_geo_2)
except:                                                #если не находим по сетевому, ищем по локальному
    geo_0 = pd.read_csv(local_path_geo_0) 
    geo_1 = pd.read_csv(local_path_geo_1)
    geo_2 = pd.read_csv(local_path_geo_2)

In [3]:
datesets = {              #словарь датасетов
    'Регион_1': geo_0,
    'Регион_2': geo_1,
    'Регион_3': geo_2
}

In [4]:
for key, val in datesets.items(): #отображение всех датасетов
    print(f'Датасет: {key}')
    display(val.head())
    val.info()
    display(val.describe())
    print()  

Датасет: Регион_1


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347



Датасет: Регион_2


...

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408



Датасет: Регион_3


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


In [5]:
#проверим на дубликаты посчитаем нулевые значения 
for key, val in datesets.items(): #отображение всех датасетов
    print(f'Датасет: {key}')
    print('Число скважин с объемом 0:', val[val['product'] == 0]['product'].count())
    print('Число дубликатов по колонке ID:', val['id'].duplicated().sum())
    print('Число дубликатов полных дубликатов:', val.duplicated().sum())

Датасет: Регион_1
Число скважин с объемом 0: 1
Число дубликатов по колонке ID: 10
Число дубликатов полных дубликатов: 0
Датасет: Регион_2
Число скважин с объемом 0: 8235
Число дубликатов по колонке ID: 4
Число дубликатов полных дубликатов: 0
Датасет: Регион_3
Число скважин с объемом 0: 1
Число дубликатов по колонке ID: 4
Число дубликатов полных дубликатов: 0


In [6]:
# Объявление функции scaler с параметром: 
# - data - датасет по региону,
# - numeric - перечень колонок, по которым необходимо провести стандартизацию данных,
#которая: 
# - разделяет исходный датасет на параметры и целевой параметр,
# - разделяет на две выборки обучающую и вариационную,
# - проводит стандартизацию данных,
# - выводит на экран проверку разделения.
# И возвращает список 4 датасетов: features_train, features_valid, target_train, target_valid.

def scaler (data, numeric):
    
    data = data.drop(['id'], axis=1) # для дальнейшей работы не нужен 
    
    #выделяем признаки и целевой признак 
    target = data['product']
    features = data.drop(['product'] , axis=1)
    
    # разделение на две выборки 
    features_train, features_valid, target_train, target_valid = train_test_split( # разделение на две выборки 
    features,
    target, 
    test_size=0.25, 
    random_state=12345,
    shuffle=True
    )
    
    # стандартизация данных
    scaler = StandardScaler()
    scaler.fit(features_train[numeric])
    features_train[numeric] = scaler.transform(features_train[numeric])
    features_valid[numeric] = scaler.transform(features_valid[numeric])
    
    # проверка разбиения
    print(features_train.shape)
    print(features_valid.shape)
    print(target_train.shape)
    print(target_valid.shape)
    
    return [features_train, features_valid, target_train, target_valid]

In [7]:
pd.options.mode.chained_assignment = None
numeric = ['f0', 'f1', 'f2']

#выполняем функцию scaler для трех регионов 
geo = []
for key, val in datesets.items(): 
    print(f'Датасет: {key}')
    geo.append(scaler(val, numeric))

Датасет: Регион_1
(75000, 3)
(25000, 3)
(75000,)
(25000,)
Датасет: Регион_2
(75000, 3)
(25000, 3)
(75000,)
(25000,)
Датасет: Регион_3
(75000, 3)
(25000, 3)
(75000,)
(25000,)


1. Провели превечный осмотр данных

- параметр product для всех трех регионов имеет 0 значения, получается есть точки где нет нефти совсем, если для 1 и 3 региона это всего 1 точка на выборку 100000 то вот для 2 региона это 8.2% от общего числа,
- проверили на дубликаты по ID они есть, но незначительны по полному кортежу дубликатов нет.

2. Провели обработку данных
- разбили на параметры и целевой параметр,
- разбили выборки на обучающие и вариационные,
- провели стандартизацию по параметрам, она ничего не дала, (проверял)  параметры f0 f1 f2 и так были в близких диапазонах. Но датасет может изменится.

## Обучение и проверка модели

In [8]:
# Объявление функции modeling с параметром: features_train, features_valid, target_train, target_valid  
#которая: 
# - обучает модель LinearRegression,
# - вычисляет предсказания,
# - проводит стандартизацию данных,
# - выводит на экран RMSE и среднее.
# И возвращает вычисленные предсказания.

def modeling(features_train, features_valid, target_train, target_valid):
    
    model = LinearRegression()
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    rmse = mean_squared_error(target_valid, predicted_valid) ** 0.5
    predicted_valid = pd.Series(predicted_valid, index=target_valid.index)
    
    print("RMSE =", rmse)
    print('Средний запас предсказанного сырья =', predicted_valid.mean())
    
    return predicted_valid

In [9]:
# выполняем функцию modeling для трех регионов 

for i in range(len(geo)): 
    print("Регион", i+1)
    geo[i].append(modeling(geo[i][0],
                        geo[i][1],
                        geo[i][2],
                        geo[i][3]))

Регион 1
RMSE = 37.5794217150813
Средний запас предсказанного сырья = 92.59256778438035
Регион 2
RMSE = 0.893099286775617
Средний запас предсказанного сырья = 68.728546895446
Регион 3
RMSE = 40.02970873393434
Средний запас предсказанного сырья = 94.96504596800489


**Выводы:**
- 1 и 3 регион имею выше значения среднее запаса предсказанного сырья чем у региона 2, что советует показателям самих дата сетов.
- Показатель RMSE для региона 2 на много лучше чем у 1 и 3 региона, предсказания на этом регионе получилось очень точные, в отличии от 1 и 3, где этот показатель 37 и 40 единиц соотвественно, и это может сказать о высоком риске при выборе этих регионов.

## Подготовка к расчёту прибыли

In [10]:
# ключевые переменные

development_budget = 1e10 # бюджет на разработку скважин в регионе
revenue_per_thousand_barrel = 4.5e5 # доход с каждой единицы продукта, за тысячу баррелей 
develop_point_number = 200 # количество точек для разработки
research_point_number = 500 # количество исследуемых точек
beta = .95 # доверительный интервал

In [11]:
min_mean = development_budget / (develop_point_number * revenue_per_thousand_barrel)
print(f'Минималньное среднее количевто продукта: {min_mean: .3f}')

Минималньное среднее количевто продукта:  111.111


**Выводы:**

- Минимальное среднее 111 тыс. баррелей.
- Ни один из регионов не имеет такого среднего показателя.
- К прогнозированию 200 лучших скважин нужно подходить очень тщательно.

## Расчёта прибыли по выбранным скважинам

In [12]:
# Объявление функции modeling с параметром:
# - target_valid 
# - predicted - предсказания
# - development_budget - бюджет на разработку скважин в регионе 
# - revenue_per_thousand_barrel - доход с каждой единицы продукта, за тысячу баррелей 
# - develop_point_number - количество точек для разработки
# И возвращает прибыль по региону исходя из предсказаний и реальных данных.

def profit_calculation(target, 
                       predicted, 
                       development_budget, 
                       revenue_per_thousand_barrel, 
                       develop_point_number):
    
    probs_sorted = predicted.sort_values(ascending=False)
    selected = target[probs_sorted.index][:develop_point_number]
    
    return revenue_per_thousand_barrel * selected.sum() - development_budget

In [13]:
# выполняем функцию profit_calculation для трех регионов 

for i in range(len(geo)):
    pf = profit_calculation(geo[i][3],
                           geo[i][4],
                           development_budget,
                           revenue_per_thousand_barrel, 
                           develop_point_number)
    print(f'Прибыль региона {i+1}: {pf:.2f}')

Прибыль региона 1: 3320826043.14
Прибыль региона 2: 2415086696.68
Прибыль региона 3: 2710349963.60


**Выводы:**

- Все 3 региона дали положительный результат с высокой рентабельностью от 24 до 33%.
- Но это 200 лучших из 25000 точек, на практике столько не разведывается.
- 1 и 3 регион все еще в лидерах, но 1 регион несмотря на более низкое среднее за счет более точных предсказаний вырвался в лидеры. 

## Расчёт прибыли и рисков 

In [14]:
# Объявление функции modeling с параметром:
# - target_valid 
# - predicted - предсказания
# - development_budget - бюджет на разработку скважин в регионе 
# - revenue_per_thousand_barrel - доход с каждой единицы продукта, за тысячу баррелей 
# - develop_point_number - количество точек для разработки
# - beta
# - research_point_number - количество исследуемых точек
#
# И среднее значение, левую и правую границу доверительного интервала и вероятность риска.


def  bootstrap_confidence_interval(target, 
                                   predicted,
                                   development_budget,
                                   develop_point_number,
                                   research_point_number,
                                   beta,
                                   revenue_per_thousand_barrel):
    
    state = np.random.RandomState(12345)
    values = []
    
    for i in range(1000): # запускаем bootstrap
        subsample = predicted.sample(
            500, 
            replace=True, 
            random_state=state
        )
        values.append(profit_calculation(target,     # считаем прибыль 
                                         subsample,
                                         development_budget, 
                                         revenue_per_thousand_barrel, 
                                         develop_point_number))
        
    values = pd.Series(values)
    confidence_interval = st.t.interval( # доверительный интервал
    alpha=beta,
    df=len(values) - 1,
    loc=values.mean(),
    scale=values.sem()
    )
    
    risk_probability = values[values < 0].count()/len(values) # доля рисков
    
    return ['', values.mean(), confidence_interval[0], confidence_interval[1], risk_probability]  

In [15]:
data = []

# выполняем функцию bootstrap_confidence_interva для трех регионов 
for i in range(len(geo)): #отображение всех датасетов

    region = bootstrap_confidence_interval(geo[i][3],
                                 geo[i][4],
                                 development_budget,
                                 develop_point_number,research_point_number,
                                 beta,
                                 revenue_per_thousand_barrel)
    region[0] = 'Регион ' + str(i+1)
    data.append(region)

In [16]:
pd.options.display.float_format = '{:.3f}'.format

In [17]:
columns = ['регион', 'среднее', 'левая_граница', 'правая_граница', 'вероятность_убытка'] # Названия колонок

pd.DataFrame(data=data, columns=columns) # Таблица с результатами

,регион,среднее,левая_граница,правая_граница,вероятность_убытка
0,Регион 1,396164984.802,379620315.148,412709654.457,0.069
1,Регион 2,456045105.787,443147248.664,468942962.909,0.015
2,Регион 3,404403866.568,387445797.471,421361935.665,0.076


**Выводы:**

- Bootstrap все поставил на свои места, взяли 500 случайных скважин, вобрали из них 200 лучших по предсказаниям, и проверили по реальным данным. Регион 2 имеет самую высокую среднее среднее прибыли, а также единственный удовлетворяет условии порога риска всего 1.5%.
- Как и отмечалось ранее 1 и 3 регионы оказались очень рискованными.

**Регион для разработки – Регион 2** 